In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from airfoil.airfoil import Airfoil, Hole, WingSegment, Decomposer, Hinge
from airfoil.spitfire import SpitfireWing
from airfoil.machine_setup import MachineSetup
from itertools import pairwise

In [ ]:
wing = SpitfireWing(half_span=500)
section_positions = np.array([
    0,
    100,
    200,
    300,
    400,
    450,
])
airfoils = wing.create_airfoils(section_positions)

wing.plot(section_positions=section_positions)

In [ ]:
chunks_a, chunks_b = [*pairwise(airfoils)], [*reversed([*pairwise(reversed(airfoils))])]

def add_holes(afs:tuple[Airfoil,Airfoil])->tuple[Airfoil,Airfoil]:
    holes = [
        Hole(diameter_mm=5, position=np.array([ -40,8])),
        Hole(diameter_mm=5, position=np.array([  10,8])),
    ]
    a, b = afs
    a = a.with_holes(holes)
    b = b.with_holes(holes)
    return a, b

def add_airleron(
        afs:tuple[Airfoil,Airfoil],
        xp1:float,
        xp2:float,
        upper_thickness:float=2,
    )->tuple[Airfoil,Airfoil]:
    a, b = afs
    a = a.with_hinge(
        Hinge(position=[xp1, 0]),
        upper_thickness=upper_thickness
    )
    b = b.with_hinge(
        Hinge(position=[xp2, 0]),
        upper_thickness=upper_thickness
    )
    return a, b


chunks_a[0] = add_holes(chunks_a[0])
chunks_b[0] = add_holes(chunks_b[0])
chunks_a[-1] = add_airleron(chunks_a[-1], wing.flap_line(400),wing.flap_line(450))
chunks_a[-2] = add_airleron(chunks_a[-2], wing.flap_line(300),wing.flap_line(400))
chunks_b[-1] = add_airleron(chunks_b[-1], wing.flap_line(450),wing.flap_line(400))
chunks_b[-2] = add_airleron(chunks_b[-2], wing.flap_line(400),wing.flap_line(300))
airfoil_pairs_a:list[WingSegment] = [
    WingSegment(a, b, length=length)
    for (a, b), length
    in zip(
        chunks_a,
        np.diff(section_positions)
    )
]
airfoil_pairs_b:list[WingSegment] = [
    WingSegment(a, b, length=length)
    for (a, b), length
    in zip(
        chunks_b,
        np.diff(section_positions)
    )
]

In [ ]:
fig, axs = plt.subplots(len(chunks_a+chunks_b),2, figsize=(20,15))
for axr, (a,b) in zip(axs,chunks_a+chunks_b):
    a.plot(axr[0])
    b.plot(axr[1])

In [ ]:
import pyvista as pv
pt = pv.Plotter()
offset = 30
for chunk in airfoil_pairs_a:
    offset += chunk.length/2
    m = chunk.to_mesh()
    pt.add_mesh(m.translate((offset,0,0)), opacity=0.5)
    offset += chunk.length/2
offset = -30
for chunk in airfoil_pairs_b:
    offset -= chunk.length/2
    m = chunk.to_mesh()
    pt.add_mesh(m.translate((offset,0,0)), opacity=0.5)
    offset -= chunk.length/2
pt.show()

In [ ]:
from itertools import chain, repeat, count
foam_cut_list = []
for side, wing_segment in chain(
        zip(
            repeat("A"),
            airfoil_pairs_a
        ),
        zip(
            repeat("B"),
            airfoil_pairs_b
        )
    ):
    bounds = wing_segment.to_mesh().bounds
    foam_cut_list.append({
        "side"            : side,
        "foam_width_mm"  : bounds.x_max-bounds.x_min,
        "foam_depth_mm": np.ceil((bounds.y_max-bounds.y_min + 10)/10)*10,
        "foam_height_mm"    : 30 if (bounds.z_max-bounds.z_min) < 30 else 50,
        "wing_segment"    : wing_segment,
    })
segments = pd.DataFrame(foam_cut_list)
segments

In [ ]:
# seg = 0 # done
# seg = 1 # done
# seg = 2 # done
# seg = 3 # done
# seg = 4 # done
# seg = 5 # done
# seg = 6 # done
# seg = 7 # done
# seg = 8 # done

seg = 9

ms = MachineSetup(
    wing_segment = segments.loc[seg, "wing_segment"  ],
    foam_width   = segments.loc[seg, "foam_width_mm" ],
    foam_depth   = segments.loc[seg, "foam_depth_mm" ],
    foam_height  = segments.loc[seg, "foam_height_mm"],
    plane_spacing= 227,
    decomposer=Decomposer(buffer=0.5, segment_target_length=1),
    max_cut_speed_mm_s=240,
    min_cut_speed_mm_s=130,
    travel_speed=1000
).with_recentered_part()

print(
    f'please configure machine as shown:\n   Foam '
    f'{segments.loc[seg, "foam_width_mm" ]:.0f} x '
    f'{segments.loc[seg, "foam_depth_mm" ]:.0f} x '
    f'{segments.loc[seg, "foam_height_mm"]:.0f} mm'
    f'\n   Plane spacing: {ms.plane_spacing:.0f} mm'
)
state = np.array([0,0,0,0])
ms.plot(state)
None

In [ ]:
from airfoil.util.serial import CNC
cnc = CNC()

In [ ]:
cnc.home()

In [ ]:
cnc.status()

In [ ]:
cnc.metric()
cnc.set_position(0,0,0,0)
cnc.absolute()

In [ ]:
# initial square-up dont change for now
cnc.travel(
    x=110-50,y=35,
    z=99 -50,a=30.5
)

In [ ]:
# initial square-up dont change for now
cnc.travel(
    x=0,y=-0.5,
    z=0,a=-0.5,
)

In [ ]:
cnc.set_position(0,0,0,0)
cnc.absolute()

In [ ]:
cnc.travel(
    x=0,y=0,
    z=0,a=0,
)

In [ ]:
cnc.send_g1_commands(ms.instructions(record_name=f"Segment {seg}"))

In [ ]:
cnc.travel(
    x=0,y=5,
    z=0,a=5,
)
cnc.travel(
    x=0,y=0,
    z=0,a=0,
)